# Predicción de Precio de Coches - Análisis Exploratorio y Selección de Variables

Este notebook documenta el análisis exploratorio realizado para el proyecto de predicción de precios de coches. Incluye la selección de variables mediante RFE y la evaluación de multicolinealidad con VIF. Sirve como respaldo técnico del pipeline implementado en `train.py`, dentro del marco del proyecto MLOps.

## 1. Planteamiento del Problema
Geely Auto, una empresa automovilística china, desea introducirse en el mercado estadounidense. Para ello, necesita entender cómo las características de los vehículos influyen en su precio. Este análisis pretende modelar dicha relación para apoyar decisiones estratégicas y de diseño.

## 2. Objetivo del Negocio
El objetivo es construir un modelo interpretable de regresión lineal que explique el precio de un automóvil en función de sus características técnicas y categóricas. Este notebook se enfoca en la exploración de los datos y la selección justificativa de variables.

## 3. Importación de Librerías
Se importan librerías necesarias para el análisis exploratorio, la regresión lineal, y las métricas estadísticas.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm


## 4. Carga de Datos
Cargamos el dataset original y realizamos una primera limpieza básica de columnas irrelevantes.

In [ ]:
df = pd.read_csv("../data/CarPrice_Assignment.csv")
df.drop(columns=['car_ID'], inplace=True)
df['CarBrand'] = df['CarName'].apply(lambda x: x.split(' ')[0].lower())
df.drop(columns=['CarName'], inplace=True)

## 5. Corrección de Valores Erróneos
Algunas marcas están mal escritas. Esta limpieza es clave para evitar codificaciones erróneas al aplicar One-Hot Encoding.

In [ ]:
df['CarBrand'] = df['CarBrand'].replace({
    'maxda': 'mazda',
    'porcshce': 'porsche',
    'toyouta': 'toyota',
    'vokswagen': 'volkswagen',
    'vw': 'volkswagen',
    'Nissan': 'nissan'
})

## 6. Transformación de Variables Categóricas
Aplicamos One-Hot Encoding a las variables categóricas para poder usarlas en regresión lineal.

In [ ]:
df_encoded = pd.get_dummies(df, drop_first=True)

## 7. Separación de Variables Independientes y Dependiente
La variable objetivo (`price`) se separa de las variables predictoras.

In [ ]:
X = df_encoded.drop(columns=['price'])
y = df_encoded['price']

## 8. Selección de Variables con RFE
Aplicamos Recursive Feature Elimination con regresión lineal como modelo base para seleccionar las 15 variables más relevantes.

In [ ]:
model = LinearRegression()
rfe = RFE(model, n_features_to_select=15)
rfe.fit(X, y)

selected_columns = X.columns[rfe.support_]
X_rfe = X[selected_columns]
selected_columns

## 9. Evaluación de Multicolinealidad con VIF
Para asegurar que no haya redundancia excesiva entre las variables seleccionadas, se calcula el Factor de Inflación de la Varianza (VIF).

In [ ]:
vif_data = pd.DataFrame()
vif_data["Feature"] = selected_columns
vif_data["VIF"] = [variance_inflation_factor(X_rfe.values, i) for i in range(X_rfe.shape[1])]
vif_data.sort_values(by="VIF", ascending=False)

## 10. Conclusiones
Las variables seleccionadas por RFE y depuradas mediante VIF son las que se han utilizado finalmente en el script `train.py`. Este proceso garantiza un modelo lineal simple, interpretable y con bajo riesgo de multicolinealidad.